**BASIC CIFAS EXAMPLE WITH CONV**

Imports...

In [1]:
from numpy.random import seed
seed(86148)

from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization as BN
from tensorflow.keras.layers import GaussianNoise as GN
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler as LRS

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
batch_size = 100
num_classes = 10
epochs = 75

In [3]:
#### LOAD AND TRANSFORM
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [4]:
## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGN(model,filters,ishape=0):
    
  if (ishape!=0):
    model.add(Conv2D(filters, (3, 3), padding='same',
                 input_shape=ishape))
  else:
    model.add(Conv2D(filters, (3, 3), padding='same'))
  
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  
  return model


In [5]:
## DEF NN TOPOLOGY  
model = Sequential()

model=CBGN(model,32,x_train.shape[1:])
model=CBGN(model,64)
model=CBGN(model,128)
model=CBGN(model,256)
model=CBGN(model,512)

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()


Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
gaussian_noise (GaussianNois (None, 32, 32, 32)        0         
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Bat

2021-12-26 19:54:51.620293: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-26 19:54:51.620381: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
## OPTIM AND COMPILE
opt = SGD(learning_rate=0.1, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [7]:

# OPTIM AND COMPILE
opt = SGD(learning_rate=0.1, decay=1e-6)

datagen = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range = 20
)


def scheduler(epoch):
    if epoch < 25:
        return .1
    elif epoch < 50:
        return 0.01
    else:
        return 0.001


filepath_val_acc="./backup/model.best.hdf5"
checkpoint_val_acc = ModelCheckpoint(filepath_val_acc, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint_val_acc]    
    
## TRAINING
history = model.fit(
    datagen.flow(x_train, y_train,batch_size=batch_size),             
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test),
    shuffle=True,
    callbacks=[LRS(scheduler), checkpoint_val_acc]
)

2021-12-26 19:54:51.975204: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-26 19:54:51.975495: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-26 19:54:52.138602: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/75
500/500 [==============================] - ETA: 0s - loss: 1.8378 - accuracy: 0.3351

2021-12-26 19:55:21.632354: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 32s 62ms/step - loss: 1.8378 - accuracy: 0.3351 - val_loss: 1.5987 - val_accuracy: 0.4112

Epoch 00001: val_accuracy improved from -inf to 0.41120, saving model to ./backup/model.best.hdf5
Epoch 2/75
500/500 [==============================] - 31s 62ms/step - loss: 1.5619 - accuracy: 0.4263 - val_loss: 1.7754 - val_accuracy: 0.4235

Epoch 00002: val_accuracy improved from 0.41120 to 0.42350, saving model to ./backup/model.best.hdf5
Epoch 3/75
500/500 [==============================] - 31s 62ms/step - loss: 1.4329 - accuracy: 0.4784 - val_loss: 1.8291 - val_accuracy: 0.4259

Epoch 00003: val_accuracy improved from 0.42350 to 0.42590, saving model to ./backup/model.best.hdf5
Epoch 4/75
500/500 [==============================] - 31s 62ms/step - loss: 1.3346 - accuracy: 0.5164 - val_loss: 1.2585 - val_accuracy: 0.5436

Epoch 00004: val_accuracy improved from 0.42590 to 0.54360, saving model to ./backup/model.best.hdf5
Epoch 5/75
500/500 [==========


Epoch 00038: val_accuracy did not improve from 0.80730
Epoch 39/75
500/500 [==============================] - 31s 62ms/step - loss: 0.6591 - accuracy: 0.7688 - val_loss: 0.5993 - val_accuracy: 0.7983

Epoch 00039: val_accuracy did not improve from 0.80730
Epoch 40/75
500/500 [==============================] - 31s 62ms/step - loss: 0.6577 - accuracy: 0.7713 - val_loss: 0.5851 - val_accuracy: 0.8054

Epoch 00040: val_accuracy did not improve from 0.80730
Epoch 41/75
500/500 [==============================] - 31s 62ms/step - loss: 0.6541 - accuracy: 0.7717 - val_loss: 0.6300 - val_accuracy: 0.7932

Epoch 00041: val_accuracy did not improve from 0.80730
Epoch 42/75
500/500 [==============================] - 31s 62ms/step - loss: 0.6572 - accuracy: 0.7694 - val_loss: 0.5642 - val_accuracy: 0.8111

Epoch 00042: val_accuracy improved from 0.80730 to 0.81110, saving model to ./backup/model.best.hdf5
Epoch 43/75
500/500 [==============================] - 31s 62ms/step - loss: 0.6522 - accuracy

In [9]:
## TEST
score = model.evaluate(x_test, y_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(f"Test error: {((1-score[1]) * 100):.3f}%")

313/313 [==============================] - 2s 7ms/step - loss: 0.5642 - accuracy: 0.8132
Test loss: 0.5641709566116333
Test accuracy: 0.813200056552887
Test error: 18.680%
